In [2]:
import pandas as pd
import numpy as np

from typing import List

In [3]:
def df_metrics2df(df: pd.DataFrame, columns: List[str], metrics: List[str] = None):
  """
    Calcula métricas estatísticas para colunas selecionadas de um DataFrame e retorna um novo DataFrame
    com as colunas originais como índices e as métricas como colunas.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo os dados numéricos.
        columns (List[str]): Lista de nomes de colunas numéricas do DataFrame `df` para calcular as métricas.
        metrics (List[str], optional): Lista de categorias de métricas a calcular.
            Pode conter qualquer combinação dos seguintes valores:
                - "centrality": Inclui média, mediana e moda.
                - "dispersion": Inclui amplitude, variância, desvio padrão e coeficiente de variação.
                - "position": Inclui os quartis Q1, Q2 (mediana) e Q3.
                - "skew": Inclui o coeficiente de assimetria (skewness).
                - "kurtosis": Inclui a curtose.
            Se None, todas as métricas são calculadas.

    Returns:
        pd.DataFrame: DataFrame com as colunas selecionadas como índice (linhas), e as métricas como colunas.

    Observações:
        - A moda pode retornar uma lista (caso haja múltiplos valores com a mesma frequência).
        - O coeficiente de variação é calculado como (desvio padrão / média), retornando NaN se a média for zero.
  """
  if metrics is None:
    metrics = ["centrality", "dispersion", "position", "skew", "kurtosis"]

  result = {}

  for col in columns:
    col_metrics = {}

    # Tendência Central
    if "centrality" in metrics:
      col_metrics["media"] = df[col].mean()
      col_metrics["mediana"] = df[col].median()
      modas = df[col].mode().tolist()
      col_metrics["moda"] = modas if len(modas) > 1 else modas[0] if modas else np.nan

    # Dispersão
    if "dispersion" in metrics:
      col_metrics["amplitude"] = df[col].max() - df[col].min()
      col_metrics["variancia"] = df[col].var()
      col_metrics["desv_pad"] = df[col].std()
      mean = df[col].mean()
      std_dev = df[col].std()
      col_metrics["coef_var"] = std_dev / mean if mean != 0 else np.nan

    # Posição
    if "position" in metrics:
      col_metrics["q1"] = df[col].quantile(0.25)
      col_metrics["q2"] = df[col].quantile(0.50)
      col_metrics["q3"] = df[col].quantile(0.75)

    # Assimetria
    if "skew" in metrics:
      col_metrics["assimetria"] = df[col].skew()

    # Curtose
    if "kurtosis" in metrics:
      col_metrics["curtose"] = df[col].kurtosis()

    result[col] = col_metrics

  return pd.DataFrame(result).T # Transpõe o DataFrame antes de retornar

In [4]:
# Teste unitário

print("CASO 0: metrics = None")
df = pd.DataFrame({
    "A": [1, 2, 2, 4, 5],
    "B": [10, 20, 30, 40, 50]
})
resume = df_metrics2df(df, columns=["A", "B"])
display(resume)

print("CASO 1: metrics != None")
df = pd.DataFrame({
    "A": [1, 2, 2, 4, 5],
    "B": [10, 20, 30, 40, 50]
})
resume = df_metrics2df(df, columns=["A", "B"], metrics=["centrality", "kurtosis"])
display(resume)

CASO 0: metrics = None


,media,mediana,moda,amplitude,variancia,desv_pad,coef_var,q1,q2,q3,assimetria,curtose
A,2.8,2.0,2.0,4.0,2.7,1.643168,0.586846,2.0,2.0,4.0,0.518421,-1.687243
B,30.0,30.0,"[10, 20, 30, 40, 50]",40,250.0,15.811388,0.527046,20.0,30.0,40.0,0.0,-1.2


CASO 1: metrics != None


,media,mediana,moda,curtose
A,2.8,2.0,2.0,-1.687243
B,30.0,30.0,"[10, 20, 30, 40, 50]",-1.2
